In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
import os
import pathlib
import time

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Decompress the compressed reorganized "Jester" dataset in Google Colab VM

start = time.time()
!cat /content/gdrive/My\ Drive/DashGC/Reorganized/Dataset.tar.gz.?? | tar -xz
stop = time.time()
print('Decompression took', round(((stop - start) / 60), 2), 'mins')

Decompression took 8.15 mins


In [0]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [0]:
# Use Pathlib to get the path root to the dataset

train_data_root_orig = "./Dataset/Train"
train_data_root = pathlib.Path(train_data_root_orig)
print(train_data_root)

val_data_root_orig = "./Dataset/Validation"
val_data_root = pathlib.Path(val_data_root_orig)
print(val_data_root)

Dataset/Train
Dataset/Validation


In [0]:
# Prints all of the directories in validation path root

for item in val_data_root.iterdir():
    print(item)

Dataset/Validation/Rolling Hand Forward
Dataset/Validation/Doing other things
Dataset/Validation/Turning Hand Counterclockwise
Dataset/Validation/Sliding Two Fingers Down
Dataset/Validation/Zooming In With Full Hand
Dataset/Validation/Rolling Hand Backward
Dataset/Validation/Swiping Left
Dataset/Validation/Thumb Up
Dataset/Validation/Zooming Out With Full Hand
Dataset/Validation/Thumb Down
Dataset/Validation/Turning Hand Clockwise
Dataset/Validation/Pushing Two Fingers Away
Dataset/Validation/No gesture
Dataset/Validation/Sliding Two Fingers Right
Dataset/Validation/Sliding Two Fingers Left
Dataset/Validation/Sliding Two Fingers Up
Dataset/Validation/Swiping Down
Dataset/Validation/Swiping Up
Dataset/Validation/Shaking Hand
Dataset/Validation/Zooming Out With Two Fingers
Dataset/Validation/Pulling Hand In
Dataset/Validation/Pulling Two Fingers In
Dataset/Validation/Swiping Right
Dataset/Validation/Drumming Fingers
Dataset/Validation/Stop Sign
Dataset/Validation/Pushing Hand Away
Datase

In [0]:
# Sorting path list by filename (aka frame id) [first] & directory name (aka video id) [second sort key]

start = time.time()

val_image_paths = list(val_data_root.glob('*/*/*'))
val_image_paths = [str(path) for path in val_image_paths]
val_image_paths.sort(key = lambda x: (int(str(x).split('/')[-1][:-5]), int(str(x).split('/')[-2])))

train_image_paths = list(train_data_root.glob('*/*/*'))
train_image_paths = [str(path) for path in train_image_paths]
train_image_paths.sort(key = lambda x: (int(str(x).split('/')[-1][:-5]), int(str(x).split('/')[-2])))

val_image_count = len(val_image_paths)
print(val_image_count)

train_image_count = len(train_image_paths)
print(train_image_count)

stop = time.time()
print('Sorting took', round(((stop - start) / 60), 2), 'mins')

532578
4219150
Sorting took 0.74 mins


In [0]:
# Prints the first 10 paths to check if sorting was correctly done.

train_image_paths[:10]

['Dataset/Train/Doing other things/1/00009.jpg',
 'Dataset/Train/Doing other things/1/00007.jpg',
 'Dataset/Train/Doing other things/1/00008.jpg',
 'Dataset/Train/Doing other things/1/00002.jpg',
 'Dataset/Train/Doing other things/1/00001.jpg',
 'Dataset/Train/Doing other things/1/00005.jpg',
 'Dataset/Train/Doing other things/1/00003.jpg',
 'Dataset/Train/Doing other things/1/00004.jpg',
 'Dataset/Train/Doing other things/1/00006.jpg',
 'Dataset/Train/Pushing Two Fingers Away/3/00009.jpg']

In [0]:
# Prints the first 10 paths to check if sorting was correctly done.

val_image_paths[:10]

['Dataset/Validation/Thumb Down/16/00009.jpg',
 'Dataset/Validation/Thumb Down/16/00007.jpg',
 'Dataset/Validation/Thumb Down/16/00008.jpg',
 'Dataset/Validation/Thumb Down/16/00002.jpg',
 'Dataset/Validation/Thumb Down/16/00001.jpg',
 'Dataset/Validation/Thumb Down/16/00005.jpg',
 'Dataset/Validation/Thumb Down/16/00003.jpg',
 'Dataset/Validation/Thumb Down/16/00004.jpg',
 'Dataset/Validation/Thumb Down/16/00006.jpg',
 'Dataset/Validation/Zooming In With Full Hand/22/00009.jpg']

In [0]:
label_names = sorted(item.name for item in train_data_root.glob('*/') if item.is_dir())
label_names

['Doing other things',
 'Drumming Fingers',
 'No gesture',
 'Pulling Hand In',
 'Pulling Two Fingers In',
 'Pushing Hand Away',
 'Pushing Two Fingers Away',
 'Rolling Hand Backward',
 'Rolling Hand Forward',
 'Shaking Hand',
 'Sliding Two Fingers Down',
 'Sliding Two Fingers Left',
 'Sliding Two Fingers Right',
 'Sliding Two Fingers Up',
 'Stop Sign',
 'Swiping Down',
 'Swiping Left',
 'Swiping Right',
 'Swiping Up',
 'Thumb Down',
 'Thumb Up',
 'Turning Hand Clockwise',
 'Turning Hand Counterclockwise',
 'Zooming In With Full Hand',
 'Zooming In With Two Fingers',
 'Zooming Out With Full Hand',
 'Zooming Out With Two Fingers']

In [0]:
# Indexing of classification class labels

label_to_index = {name: index for index, name in enumerate(label_names)}
label_to_index

{'Doing other things': 0,
 'Drumming Fingers': 1,
 'No gesture': 2,
 'Pulling Hand In': 3,
 'Pulling Two Fingers In': 4,
 'Pushing Hand Away': 5,
 'Pushing Two Fingers Away': 6,
 'Rolling Hand Backward': 7,
 'Rolling Hand Forward': 8,
 'Shaking Hand': 9,
 'Sliding Two Fingers Down': 10,
 'Sliding Two Fingers Left': 11,
 'Sliding Two Fingers Right': 12,
 'Sliding Two Fingers Up': 13,
 'Stop Sign': 14,
 'Swiping Down': 15,
 'Swiping Left': 16,
 'Swiping Right': 17,
 'Swiping Up': 18,
 'Thumb Down': 19,
 'Thumb Up': 20,
 'Turning Hand Clockwise': 21,
 'Turning Hand Counterclockwise': 22,
 'Zooming In With Full Hand': 23,
 'Zooming In With Two Fingers': 24,
 'Zooming Out With Full Hand': 25,
 'Zooming Out With Two Fingers': 26}

In [0]:
# Stores the label index for each video

start = time.time()

train_image_labels = [label_to_index[pathlib.Path(path).parent.parent.name]
                    for path in train_image_paths]

val_image_labels = [label_to_index[pathlib.Path(path).parent.parent.name]
                    for path in val_image_paths]

print("First 10 labels indices: ", train_image_labels[:10])

stop = time.time()
print('Labeling took', round(((stop - start) / 60), 2), 'mins')

First 10 labels indices:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 6]
Labeling took 0.5 mins


In [0]:
# Decode the video frames as int64 values of color & recast it as float32
# Resize the video frames into a squared size (100, 100, channels = 3)

#@tf.function
def preprocess_image(image):
    '''
    Decodes the binary form to normalized vector (aka tensor) form
    '''
    # img_raw = tf.io.read_file(img_path)
    # img_tensor = tf.image.decode_image(img_raw)
    image = tf.image.decode_jpeg(image, channels=3) # imports RGB channels, jpeg doesn't have a 4th channel alpha for transparency like png
    image = tf.image.resize(image, [100, 100]) # original dimensions are (176, 100)
    image /= 255.0  # normalize to [0,1] range
    return image

def load_and_preprocess_image(path):
    '''
    Open the file from path as binary
    '''
    image = tf.io.read_file(path)
    return preprocess_image(image)

def load_and_preprocess_from_path_label(path, label):
    return load_and_preprocess_image(path), label

In [0]:
# Load the images as tensors (numpy-like multidimensional arrays)

train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)

val_path_ds = tf.data.Dataset.from_tensor_slices(val_image_paths)
val_image_ds = val_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)

In [0]:
# Labeling

train_label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(train_image_labels, tf.int64))
val_label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(val_image_labels, tf.int64))

train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
val_image_label_ds = tf.data.Dataset.zip((val_image_ds, val_label_ds))

In [0]:
print(type(train_image_label_ds), type(val_image_label_ds))

<class 'tensorflow.python.data.ops.dataset_ops.ZipDataset'> <class 'tensorflow.python.data.ops.dataset_ops.ZipDataset'>


In [0]:
tf.keras.layers.Input(shape = (100, 100, 3), sparse = True)

In [0]:
img_raw = tf.io.read_file("Dataset/Validation/Thumb Down/16/00001.jpg")
img_tensor = tf.image.decode_image(img_raw)

print(img_tensor.shape)
print(img_tensor.dtype)

img_final = tf.image.resize(img_tensor, [100, 100])
img_final = img_final/255.0
print(img_final.shape)
print(img_final.numpy().min())
print(img_final.numpy().max())
print(img_final)

(100, 176, 3)
<dtype: 'uint8'>
(100, 100, 3)
0.0
0.9341177
tf.Tensor(
[[[0.35929412 0.41811767 0.3985098 ]
  [0.35686275 0.41568628 0.39607844]
  [0.35686275 0.41568628 0.39607844]
  ...
  [0.10235296 0.13647054 0.11725488]
  [0.09913725 0.12266666 0.10698039]
  [0.09411765 0.11764706 0.10196079]]

 [[0.35835293 0.41717646 0.3975686 ]
  [0.3602353  0.41905883 0.399451  ]
  [0.3498039  0.40862745 0.3890196 ]
  ...
  [0.09058826 0.12470584 0.10549017]
  [0.0907451  0.11427451 0.09858824]
  [0.09262743 0.11615685 0.10047057]]

 [[0.35835293 0.41717646 0.3975686 ]
  [0.3602353  0.41905883 0.399451  ]
  [0.34627452 0.40509805 0.3854902 ]
  ...
  [0.07843138 0.12078427 0.09764703]
  [0.07843138 0.11372549 0.09411765]
  [0.08721565 0.12250976 0.10290192]]

 ...

 [[0.00392157 0.01568628 0.        ]
  [0.00447059 0.01623529 0.        ]
  [0.01843137 0.02313725 0.        ]
  ...
  [0.02588226 0.02980383 0.01058816]
  [0.0050196  0.00894117 0.        ]
  [0.00635292 0.01027449 0.        ]]

 [[0